In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from collections import defaultdict
%matplotlib inline

In [5]:
df2 = pd.read_csv("csv/20151016135827-NAV-GBR.csv", parse_dates=['Date', 'LocalTime'], index_col='LocalTime')
df2.head()

,Boat,Date,Secs,Zone,Lat,Lon,Hdg,Heel,Pitch,COG,...,yHdg,ySpeed,yTWS,yTWD,yAWS,yAWA,yTWA,ySOG,yCOG,yRudder
LocalTime,,,,,,,,,,,,,,,,,,,,,
2017-05-09 13:58:28.404,GBR,16:10:2015,50308.404,-3,32.299371,-64.825425,30.3,0.5,-0.9,15.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-09 13:58:28.604,GBR,16:10:2015,50308.604,-3,32.299373,-64.825425,30.5,0.5,-0.8,10.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-09 13:58:28.804,GBR,16:10:2015,50308.804,-3,32.299374,-64.825424,30.9,0.4,-0.6,6.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-09 13:58:29.004,GBR,16:10:2015,50309.004,-3,32.299376,-64.825424,31.3,0.5,-0.3,6.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-09 13:58:29.204,GBR,16:10:2015,50309.204,-3,32.299377,-64.825424,31.7,0.6,0.0,8.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df2.columns

Index([u'Boat', u'Date', u'Secs', u'Zone', u'Lat', u'Lon', u'Hdg', u'Heel',
       u'Pitch', u'COG', u'SOG', u'CourseWindDirection', u'CourseWindSpeed',
       u'yHdg', u'ySpeed', u'yTWS', u'yTWD', u'yAWS', u'yAWA', u'yTWA',
       u'ySOG', u'yCOG', u'yRudder'],
      dtype='object')

In [7]:
nav_files = glob.glob('csv/[0-9]*-NAV*.*')
nav_files[:5]

['csv/20151016135827-NAV-FL1.csv',
 'csv/20151016135827-NAV-FL2.csv',
 'csv/20151016135827-NAV-FM1.csv',
 'csv/20151016135827-NAV-FM2.csv',
 'csv/20151016135827-NAV-FRA.csv']

In [8]:
race_dict = defaultdict(list)
for f in nav_files:
    race_dict[f.split('csv/')[1].split('-')[0]].append(f)

In [9]:
df_dict = {}
off_df_dict = {}
comp_df_dict = {}
race_boats = ['USA', 'SWE', 'GBR', 'FRA', 'NZL', 'JPN']
for race_date, race_files in race_dict.items():
    for rf in race_files[:]:
        boat_name = rf.split('-NAV-')[1].replace('.csv', '')
        t_df = pd.read_csv(
            rf, parse_dates=['LocalTime'], index_col='LocalTime'
        )
        t_df.drop('Zone', axis=1, inplace=True)
        t_df.drop('Boat', axis=1, inplace=True)
        t_df.drop('Date', axis=1, inplace=True)
        t_df = t_df.resample('200ms').first()
        df_dict[boat_name]  = t_df
        if boat_name in race_boats:
            comp_df_dict[boat_name] = t_df
        else:
            off_df_dict[boat_name] = t_df
    break

In [10]:
comp_df = pd.concat(comp_df_dict.values(), axis=1, keys=comp_df_dict.keys())
off_df = pd.concat(off_df_dict.values(), axis=1, keys=off_df_dict.keys())
#full_dct = {'competitor': comp_df, 'official': off_df}
full_df = pd.concat([comp_df, off_df], axis=1, keys=['competitor', 'official'])
full_df = full_df.swaplevel(1,2, axis=1)
!rm race1.hd5
comp_df.to_hdf('race1.hd5', 'nav', complevel=9, complib='blosc')
full_df.head()

competitor                                          \
                              Secs        Lat        Lon    Hdg Heel Pitch   
                               FRA        FRA        FRA    FRA  FRA   FRA   
LocalTime                                                                    
2017-05-09 14:49:18.400        NaN        NaN        NaN    NaN  NaN   NaN   
2017-05-09 14:49:18.600  53358.605  32.300481 -64.826227  122.7 -0.3  -1.1   
2017-05-09 14:49:18.800  53358.805  32.300479 -64.826224  122.2 -0.3  -1.0   
2017-05-09 14:49:19.000  53359.005  32.300476 -64.826220  121.7 -0.3  -0.9   
2017-05-09 14:49:19.200  53359.205  32.300473 -64.826216  121.2 -0.2  -0.8   

                                                                          \
                           COG   SOG CourseWindDirection CourseWindSpeed   
                           FRA   FRA                 FRA             FRA   
LocalTime                                                                  
2017-05-09 14:49:18.400    NaN   NaN                 NaN             NaN   
2017-05-09 14:49:18.600  130.5  4.35                47.8             3.9   
2017-05-09 14:49:18.800  129.9  4.35                48.2             3.9   
2017-05-09 14:49:19.000  129.3  4.35                49.1             3.9   
2017-05-09 14:49:19.200  128.7  4.39                49.9             3.9   

                          ...   official                                       \
                          ...       yHdg ySpeed yTWS yTWD yAWS yAWA yTWA ySOG   
                          ...         U2     U2   U2   U2   U2   U2   U2   U2   
LocalTime                 ...                                                   
2017-05-09 14:49:18.400   ...        0.0    4.1  0.0  0.0  0.0  0.0  0.0  0.0   
2017-05-09 14:49:18.600   ...        NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2017-05-09 14:49:18.800   ...        NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2017-05-09 14:49:19.000   ...        0.0    5.7  0.0  0.0  0.0  0.0  0.0  0.0   
2017-05-09 14:49:19.200   ...        NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN   

                                      
                        yCOG yRudder  
                          U2      U2  
LocalTime                             
2017-05-09 14:49:18.400  0.0     0.0  
2017-05-09 14:49:18.600  NaN     NaN  
2017-05-09 14:49:18.800  NaN     NaN  
2017-05-09 14:49:19.000  0.0     0.0  
2017-05-09 14:49:19.200  NaN     NaN  

[5 rows x 460 columns]